How the simulation is done:
- First, a base simulation is done per sample size.
- Then, as needed, more simulations are done by upticking the iteration values.

# Imports:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression

from scipy.stats import chi2
from collections import Counter

from itertools import permutations
from tqdm import tqdm, trange

In [2]:
import pickle
import statsmodels.api as sm
from collections import defaultdict
from scipy.special import expit, logit

# Helper Functions:

In [3]:
import sys

sys.path.append('../')

from ddc_utils import *
from data_generating_utils import *

In [4]:
def is_binomial_data_seperable(
    df: pd.DataFrame, binary_col: str, cont_col: str
) -> bool:
    """
    Given a dataframe, column for binary rv, and column for continuous rv, returns True if the
        binary rv is seperable.
    """
    separability_check_df = (
        df[[binary_col, cont_col]].groupby(binary_col)[cont_col].agg(["min", "max"])
    )

    # checks if there is y = 0, y = 1, as well as whether min(y = 0) > max(y = 1), and vice versa
    return (
        (len(separability_check_df) < 2)
        or (separability_check_df.iloc[0, 0] > separability_check_df.iloc[1, 1])
        or (separability_check_df.iloc[1, 0] > separability_check_df.iloc[0, 1])
    )

def to_pickle_obj(file_path, raw_data):
    with open(file_path, "wb") as handle:
        pickle.dump(raw_data, handle)

def read_pickle_obj(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# Hyperparams:

In [5]:
pop_index = 1
iter_val = 4

In [6]:
rand_generator = np.random.default_rng(seed=333 * pop_index + iter_val)

In [7]:
population_size = 100_000
number_of_coefficients = 1

num_iters_per_population_for_small_samples = 50_000
num_iters_per_population_for_large_samples = 5_000
small_large_sample_co = 100

# biased sampling scheme params:
sample_probability_centering = 0.77
sample_probability_bias_factor = 1

In [8]:
# iter 0:
# ALL_SAMPLE_SIZES = (
#     [i for i in range(35, 45)]
#     + [50, 70, 100, 150, 250, 400, 600, 1000, 1400]
#     + [2000, 3000, 5000, 7500, 10_000, 15_000, 20_000]
# )


# iter 2-4:
ALL_SAMPLE_SIZES = (
    [30, 31, 33, 35, 37, 39, 40]
    + [50, 70, 100, 150, 200, 250, 300, 350, 400, 500, 600, 700, 800, 900, 1000, 1400]
)


# Load Finite Population Data:

In [9]:
pickle_filename = f'base_population_data_Logit_1.pickle'
pop_data = pd.read_pickle(pickle_filename)

# Run:

In [10]:
feature_cols = [f'x_{i}' for i in range(number_of_coefficients)]

In [11]:
sample_specific_non_separable_count = {}

#### get population-level statistics:

In [12]:
pop_x = pop_data[feature_cols]
pop_y = pop_data['y']

pop_model = sm.Logit(endog = pop_y, exog = pop_x).fit(disp=0)
pop_beta = np.array(pop_model.params)
pop_gs = pop_x * (np.array(pop_y).reshape((population_size, 1)) - \
              np.array(pop_model.predict()).reshape((population_size, 1)))

#### actually run:

In [13]:
for temp_sample_size in tqdm(ALL_SAMPLE_SIZES):
    # where the data will be saved:
    all_jns_biased, all_ddc_biased, all_sample_beta_biased = [], [], []
    all_jns_full, all_ddc_full, all_sample_beta_full = [], [], []

    all_realized_sample_sizes = []

    # set up how much to sample for this population:
    non_separable_count = 0
    if temp_sample_size < small_large_sample_co:
        num_iters_per_population = num_iters_per_population_for_small_samples
    else:
        num_iters_per_population = num_iters_per_population_for_large_samples

    for _ in trange(num_iters_per_population, mininterval=10):
        """
            First, sample a valid dataset (which is not seperable):
        """
        # use sampling scheme to sample data:
        obtained_valid_sample = False

        while not obtained_valid_sample:
            # intended sample:
            pop_data["r0"] = 0
            pop_data.loc[
                np.random.choice(pop_data.index, size=temp_sample_size, replace=False),
                "r0",
            ] = 1

            full_sampled_data = pop_data[pop_data["r0"] == 1]
            
            # biased sample:
            pop_data["r"] = 0
            
            marginal_probabilities = expit(
                logit(sample_probability_centering)
                + sample_probability_bias_factor
                * (2 * full_sampled_data["y"] - 1)
                * full_sampled_data["x_0"]
            )
            other_sample_indices = marginal_probabilities.index[
                rand_generator.binomial(n=1, p=marginal_probabilities) == 1
            ]
            pop_data.loc[other_sample_indices, "r"] = 1

            # sample_data here means the biased sample data.
            sample_data = pop_data[pop_data["r"] == 1]

            # if the sample size is too small, check for seperability:
            realised_sample_size = len(other_sample_indices)
            if realised_sample_size < 1_000:
                if is_binomial_data_seperable(sample_data, 'y', 'x_0'):
                    non_separable_count = non_separable_count + 1
                    continue
            
            obtained_valid_sample = True
        
        """
            Then, compute the logistic betas, ddc, Jns:
        """
        # compute biased x, y, model, beta
        sample_x, sample_y = sample_data[feature_cols], sample_data["y"]
        sample_beta = np.array(
            sm.Logit(endog=sample_y, exog=sample_x).fit(disp=0, maxiter=5_00).params
        )
        sample_r = pop_data["r"]

        # compute full x, y, model, beta
        sample_x_full, sample_y_full = (
            full_sampled_data[feature_cols],
            full_sampled_data["y"],
        )
        sample_beta_full = np.array(
            sm.Logit(endog=sample_y_full, exog=sample_x_full)
            .fit(disp=0, maxiter=5_00)
            .params
        )
        sample_r_full = pop_data["r0"]

        # compute biased versions of things:
        all_sample_beta_biased.append(pd.Series(sample_beta))
        all_ddc_biased.append(pop_gs.corrwith(sample_r)[["x_0"]])
        all_jns_biased.append(
            compute_average_jn(
                pop_beta, sample_beta, sample_x, sample_y, model_type="Logit"
            )
        )

        all_realized_sample_sizes.append(realised_sample_size)

        # compute full versions of things:
        all_sample_beta_full.append(pd.Series(sample_beta_full))
        all_ddc_full.append(pop_gs.corrwith(sample_r_full)[["x_0"]])
        all_jns_full.append(
            compute_average_jn(
                pop_beta,
                sample_beta_full,
                sample_x_full,
                sample_y_full,
                model_type="Logit",
            )
        )

    sample_specific_non_separable_count[temp_sample_size] = non_separable_count

    """
        Save the data!
    """
    # concatenate the biased versions:
    temp_samp_beta_biased = pd.concat(all_sample_beta_biased, axis=1).T
    temp_ddc_biased = pd.concat(all_ddc_biased, axis=1).T
    temp_jn_biased = pd.concat(all_jns_biased, axis=1).T.reset_index(drop=True)
    realised_sizes = pd.Series(all_realized_sample_sizes)

    # concat the SRS versions:
    temp_samp_beta_full = pd.concat(all_sample_beta_full, axis=1).T
    temp_ddc_full = pd.concat(all_ddc_full, axis=1).T
    temp_jn_full = pd.concat(all_jns_full, axis=1).T.reset_index(drop=True)

    temp_ss_data = pd.concat(
        [
            temp_samp_beta_biased,
            temp_ddc_biased,
            temp_jn_biased,
            realised_sizes,
            temp_samp_beta_full,
            temp_ddc_full,
            temp_jn_full,
        ],
        axis=1,
    )
    temp_ss_data.columns = [
        "samp_biased",
        "ddc_biased",
        "jn_biased",
        "realized_size_biased",
        "samp_intended",
        "ddc_intended",
        "jn_intended",
    ]

    temp_ss_data["sample_size"] = temp_sample_size
    temp_ss_data["pop_beta"] = pop_beta[0]

    temp_ss_data["mse_biased"] = (
        temp_ss_data["pop_beta"] - temp_ss_data["samp_biased"]
    ) ** 2

    temp_ss_data["mse_intended"] = (
        temp_ss_data["pop_beta"] - temp_ss_data["samp_intended"]
    ) ** 2

    to_pickle_obj(f"sim_results/sim_{temp_sample_size}_iter_{iter_val}.pickle", temp_ss_data)

  0%|                                                                                           | 0/23 [00:00<?, ?it/s]
%|                                                                                        | 0/50000 [00:00<?, ?it/s]
%|█▌                                                                            | 972/50000 [00:10<08:24, 97.17it/s]
%|███                                                                         | 1994/50000 [00:20<07:59, 100.12it/s]
%|███                                                                         | 1994/50000 [00:30<07:59, 100.12it/s]
%|████▌                                                                       | 3005/50000 [00:30<07:47, 100.48it/s]
%|██████▏                                                                     | 4067/50000 [00:40<07:27, 102.73it/s]
%|██████▏                                                                     | 4067/50000 [00:50<07:27, 102.73it/s]
%|███████▋                                                   

 21%|████████████████▍                                                            | 2136/10000 [02:53<10:46, 12.17it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 23%|█████████████████▍                                                           | 2263/10000 [03:03<10:29, 12.30it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 24%|██████████████████▍                                                          | 2389/10000 [03:15<10:48, 11.74it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: 

 46%|███████████████████████████████████▍                                         | 4606/10000 [06:31<08:02, 11.19it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 48%|█████████████████████████████████████▏                                       | 4828/10000 [06:51<07:51, 10.98it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 49%|█████████████████████████████████████▉                                       | 4935/10000 [07:01<07:45, 10.87it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: 


 75%|█████████████████████████████████████████████████████████▊                   | 7510/10000 [11:00<03:41, 11.23it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 78%|████████████████████████████████████████████████████████████▍                | 7843/10000 [11:31<03:17, 10.93it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.

In [14]:
temp_ss_data.describe()

,samp_biased,ddc_biased,jn_biased,realized_size_biased,samp_intended,ddc_intended,jn_intended,sample_size,pop_beta,mse_biased,mse_intended
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5.000000e+03,5000.000000,5.000000e+03
mean,1.521020,0.017120,0.120167,1123.358200,1.011313,-0.000072,0.143971,1400.0,1.010783e+00,0.269238,4.963701e-03
std,0.094327,0.002544,0.005481,15.008029,0.070459,0.003183,0.005580,0.0,2.220668e-16,0.098586,7.057778e-03
min,1.162610,0.006178,0.100422,1066.000000,0.787433,-0.011733,0.122373,1400.0,1.010783e+00,0.023051,5.799448e-11
25%,1.455346,0.015405,0.116503,1113.000000,0.963509,-0.002198,0.140098,1400.0,1.010783e+00,0.197637,5.044485e-04
50%,1.518398,0.017148,0.120080,1123.000000,1.010032,-0.000034,0.143960,1400.0,1.010783e+00,0.257673,2.296789e-03
75%,1.583271,0.018850,0.123748,1134.000000,1.059186,0.002150,0.147777,1400.0,1.010783e+00,0.327743,6.591521e-03
max,1.870004,0.026127,0.144001,1187.000000,1.286085,0.011456,0.167087,1400.0,1.010783e+00,0.738261,7.579127e-02
